## Data Acquisition

[Dataset](https://www.kaggle.com/datasets/chethuhn/network-intrusion-dataset)

In [1]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d chethuhn/network-intrusion-dataset
!unzip network-intrusion-dataset.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/chethuhn/network-intrusion-dataset
License(s): CC0-1.0
network-intrusion-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  network-intrusion-dataset.zip
replace Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Friday-WorkingHours-Morning.pcap_ISCX.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Monday-WorkingHours.pcap_ISCX.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Tuesday-WorkingHours.pcap_ISCX.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Wednesday-workingHours.pcap_ISCX.c

## Import libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score

In [3]:
df=pd.read_csv('Friday-WorkingHours-Morning.pcap_ISCX.csv')
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,3268,112740690,32,16,6448,1152,403,0,201.5,204.724205,...,32,3.594286e+02,1.199802e+01,380,343,16100000.0,4.988048e+05,16400000,15400000,BENIGN
1,389,112740560,32,16,6448,5056,403,0,201.5,204.724205,...,32,3.202857e+02,1.574499e+01,330,285,16100000.0,4.987937e+05,16400000,15400000,BENIGN
2,0,113757377,545,0,0,0,0,0,0.0,0.000000,...,0,9.361829e+06,7.324646e+06,18900000,19,12200000.0,6.935824e+06,20800000,5504997,BENIGN
3,5355,100126,22,0,616,0,28,28,28.0,0.000000,...,32,0.000000e+00,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
4,0,54760,4,0,0,0,0,0,0.0,0.000000,...,0,0.000000e+00,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN


In [4]:
df[' Label'].value_counts()

,count
Label,
BENIGN,189067
Bot,1966


In [5]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [6]:
df[' Label'].value_counts()

,count
Label,
BENIGN,188955
Bot,1956


In [7]:
df.isnull().sum()

,0
Destination Port,0
Flow Duration,0
Total Fwd Packets,0
Total Backward Packets,0
Total Length of Fwd Packets,0
...,...
Idle Mean,0
Idle Std,0
Idle Max,0
Idle Min,0


In [8]:
df.isna().sum()

,0
Destination Port,0
Flow Duration,0
Total Fwd Packets,0
Total Backward Packets,0
Total Length of Fwd Packets,0
...,...
Idle Mean,0
Idle Std,0
Idle Max,0
Idle Min,0


In [9]:
from imblearn.over_sampling import SMOTE

x = df.drop(' Label', axis=1)
y = df[' Label']

smote = SMOTE()
x, y = smote.fit_resample(x, y)

## Scaling values


In [10]:
scaler = StandardScaler()
cols = x.columns
x = pd.DataFrame(scaler.fit_transform(x), columns=cols)
x.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,-0.477537,4.727147,0.030014,0.005745,0.339905,-0.005613,0.069796,-0.414661,0.272422,0.289463,...,0.012023,1.498991,-0.069681,-0.077395,-0.097000,-0.051424,1.402808,0.265826,1.389741,1.360858
1,-0.631275,4.727141,0.030014,0.005745,0.339905,-0.003957,0.069796,-0.414661,0.272422,0.289463,...,0.012023,1.498991,-0.069744,-0.077384,-0.097052,-0.051530,1.402808,0.265819,1.389741,1.360858
2,-0.652047,4.772193,0.687278,-0.009462,-0.114975,-0.006102,-0.175715,-0.414661,-0.190117,-0.206671,...,-0.008145,-4.368608,15.060685,20.760270,19.741431,-0.052020,1.015263,4.436237,1.815845,0.363660
3,-0.366092,-0.263575,0.017202,-0.009462,-0.071518,-0.006102,-0.158658,0.466580,-0.125844,-0.206671,...,0.020091,1.498991,-0.070262,-0.077429,-0.097399,-0.052055,-0.197059,-0.057340,-0.198463,-0.191122
4,-0.652047,-0.265585,-0.005860,-0.009462,-0.114975,-0.006102,-0.175715,-0.414661,-0.190117,-0.206671,...,-0.008145,-4.368608,-0.070262,-0.077429,-0.097399,-0.052055,-0.197059,-0.057340,-0.198463,-0.191122


## Encoding

In [11]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

## Significant Features

In [12]:
from sklearn.ensemble import RandomForestClassifier

model0 = RandomForestClassifier()
model0.fit(x, y)

feat_importances = pd.Series(model0.feature_importances_, index=x.columns)
significant_features = feat_importances.nlargest(5)
print(significant_features)

 Destination Port           0.146745
 Avg Bwd Segment Size       0.087717
 Bwd Packet Length Mean     0.071138
 Init_Win_bytes_backward    0.068982
Init_Win_bytes_forward      0.064572
dtype: float64


In [13]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=12)

## Machine Learning Approach

In [14]:
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(max_iter=1000)
model1.fit(xtrain, ytrain)

ypreds = model1.predict(xtest)
acc1 = accuracy_score(ytest, ypreds)
print(acc1)
report = classification_report(ytest, ypreds)
print(report)

0.9715540737212564
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     37725
           1       0.96      0.99      0.97     37857

    accuracy                           0.97     75582
   macro avg       0.97      0.97      0.97     75582
weighted avg       0.97      0.97      0.97     75582



In [15]:
from sklearn.tree import DecisionTreeClassifier

model3 = DecisionTreeClassifier()
model3.fit(xtrain, ytrain)

ypreds = model3.predict(xtest)
acc3 = accuracy_score(ytest, ypreds)
print(acc3)
report = classification_report(ytest, ypreds)
print(report)

0.9993252361673415
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     37725
           1       1.00      1.00      1.00     37857

    accuracy                           1.00     75582
   macro avg       1.00      1.00      1.00     75582
weighted avg       1.00      1.00      1.00     75582



In [16]:
from sklearn.ensemble import RandomForestClassifier

model4 = RandomForestClassifier()
model4.fit(xtrain, ytrain)

ypreds = model4.predict(xtest)
acc4 = accuracy_score(ytest, ypreds)
print(acc4)
report = classification_report(ytest, ypreds)
print(report)

0.9996427720885925
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     37725
           1       1.00      1.00      1.00     37857

    accuracy                           1.00     75582
   macro avg       1.00      1.00      1.00     75582
weighted avg       1.00      1.00      1.00     75582



In [17]:
from xgboost import XGBClassifier

model5 = XGBClassifier()
model5.fit(xtrain, ytrain)

ypreds = model5.predict(xtest)
acc5 = accuracy_score(ytest, ypreds)
print(acc5)
report = classification_report(ytest, ypreds)
print(report)

0.9995898494350507
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     37725
           1       1.00      1.00      1.00     37857

    accuracy                           1.00     75582
   macro avg       1.00      1.00      1.00     75582
weighted avg       1.00      1.00      1.00     75582

